In [1]:
#Importing essentials libraries

import csv
import numpy
import matplotlib.pyplot as plt
import pandas
import math

import keras
from keras.models import Sequential
from keras.layers import Dense,Activation, Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import collections
import datetime
import scipy

import graphviz
import pydot
import plotly
import plotly.plotly as py
from plotly.graph_objs import Scatter, Heatmap, Layout
plotly.offline.init_notebook_mode(connected=True)

ModuleNotFoundError: No module named 'keras'

In [ ]:
#Function for building the model

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
  return numpy.array(dataX), numpy.array(dataY)

In [ ]:
#Read dataset file
file = pandas.read_csv("improved_spvm_2015-2019.csv")

In [ ]:
#Display Dataset to see what we are working on
jours = file["JOUR"]
jours = list(jours)
jours = collections.Counter(jours)

jours_x = list(jours.keys())
jours_y = list(jours.values())

plotly.offline.iplot({
    "data": [Scatter(x= jours_x, y= jours_y)],
    "layout": Layout(title="Crimes per day")
})


In [ ]:
#Preparing the dataset : 70% trainset ; 30% test set.

#x = int(0.70*len(jours_x))
y = int(0.70*len(jours_y))

scaler = MinMaxScaler(feature_range=(0,1))

x_train = jours_y[0:y]
x_test = jours_y[y+1:-1]

X = len(x_train)
Y = len(x_test)

x_train = numpy.asarray(x_train)
x_test = numpy.asarray(x_test)

x_train = x_train.reshape(X,1)
x_test = x_test.reshape(Y,1)

#Nu-Vac
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

print("Preparing dataset finished !")

In [ ]:
#Building the training models

look_back = 14 #Numbers of days to look back and feed the model

trainX, trainY = create_dataset(x_train, look_back=look_back)
testX, testY = create_dataset(x_test, look_back=look_back)

trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


tbCallBack = keras.callbacks.TensorBoard(log_dir='Graph', histogram_freq=0,  
          write_graph=True, write_images=True)

model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))

model.summary()

model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae'])
history = model.fit(trainX, trainY, epochs=100, batch_size=14, verbose=0,shuffle=True,callbacks=[tbCallBack])

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
#Training Metrics

# Create traces
trace0 = Scatter(
    y = history.history['loss'],
    mode = 'markers+lines',
    name = 'Train'
)
#trace1 = go.Scatter(
#    y = history.history['val_loss'],
#    mode = 'markers+lines',
#    name = 'Test'
#)

data = [trace0]

plotly.offline.iplot(data, filename='Loss+ValLoss')

trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:, 0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:, 0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
#Showing Predictions

predictions = (scaler.inverse_transform(testPredict)).flatten()
normal_vals = (scaler.inverse_transform(x_test)).flatten()
normal_vals = normal_vals[14:-1]

random_x = jours_x[y+1:-1]


# Create traces
trace_pred = Scatter(
    x = random_x,
    y = predictions.tolist(),
    mode = 'markers',
    name = 'Predictions'
)
trace_normal = Scatter(
    x = random_x,
    y = normal_vals.tolist(),
    mode = 'markers',
    name = 'Actual Values'
)

data = [trace_pred, trace_normal]

plotly.offline.iplot(data, filename='Res')